In [58]:
import pandas as pd
import pandas_datareader.data as pdr
import datetime
import copy
import matplotlib.pyplot as plt
import yahoofinance as yf

In [59]:
ticker="^GSPC"
SnP=pdr.get_data_yahoo(ticker,datetime.date.today()-datetime.timedelta(1825),datetime.date.today())

In [48]:
def CAGR(DF):
    df = DF.copy()
    df["cumulative_return"]=(1+df["mon_ret"]).cumprod()
    n=len(df)/12   #Applying CAGR
    CAGR = (df["cumulative_return"].tolist[-1])**(1/n)-1
    return CAGR

In [49]:
def volatility(DF):
    df=DF.copy()
    vol=df["return"].std()*np.sqrt(12)
    return volatility

In [50]:
def sharpe(DF,Risk_free_rate):
    df=DF.copy()
    sr=(CAGR(df)-Risk_free_rate)/volatility(df)
    return sharpe

In [51]:
def max_dd(DF):
    df=DF.copy()
    df["cum_return"]=(1+df["return"]).cumprod()
    df["cum_roll_max"]=df["cum_return"].cummax()
    df["drawdown"]=df["cum_roll_max"]-df["cum_return"]
    max_dd=(df["drawdown"]/df["cum_roll_max"]).max()
    return max_dd

In [ ]:
#DJI Constituent stocks

In [37]:
tickers=["MMM","AXP","AMGN","AAPL","BA","CAT","CVX","CSCO","KO","GS","HD","HON","INTC","IBM","JNJ","JPM","MCD","MRK","MSFT","NKE","PG","CRM","TRV","UNH","VZ","V","WBA","WMT","DIS"]

In [38]:
ohlc_mon={} #directory

In [39]:
attempt = 0 #passthrough variable

In [40]:
drop=[] #list initialization

In [ ]:
while len(tickers) != 0 and attempt <= 5:
    tickers = [j for j in tickers if j not in drop] # remove stocks after data extraction
    for i in range(len(tickers)):
        try:
            ohlc_mpn[tickers[i]]=pdr.get_data_yahoo(tickers[i],datetime.date.today().datetime.timedelta)
            ohlc_mon[tickers[i]].dropna(inplace=True)
            drop.append(tickers[i])
        except:
            print(tickers[i],";failed to fetch data...retrying")
            continue
        attempt+=1
tickers = ohlc_mon.keys()

In [42]:
ohlc_dict=copy.deepcopy(ohlc_mon)

In [43]:
return_df=pd.DataFrame()

In [ ]:
for ticker in tickers:
    print("calculating monthly return for",ticker)
    ohlc_dict[ticker]["mon_ret"]=ohlc_dict[ticker]["Adj Close"].pct_change()
    return_df[ticker]=ohlc_dict[ticker]["mon_ret"]

In [53]:
DF=return_df
n=6
x=3
portfolio=["MMM","AXP","AMGN","AAPL","BA","CAT"]

In [55]:
DF=return_df
n=6
x=3
def pflio(DF,n,x):
    #Returns cumulative portfolio return
    #DF = Dataframe with monthly return info for all stocks
    #n = number of stock in the portfolio
    #x = number of underperforming stocks to be removed from portfolio monthly
    df=DF.copy()
    portfolio=[]
    monthly_ret=[0]
    for i in range(1,len(df)):
        if len(portfolio)>0:
            monthly_ret.append(df[portfolio].iloc[i,:].mean())
            bad_stocks=df[portfolio].iloc[i,:].sort_values(ascending=True)[:x].index.values.tolist()
            portfolio=[t for t in portfolio if t not in bad_stocks]
        fill = m - len(portfolio)
        new_picks=df[[t for t in tickers if t not in portfolio]].iloc[i,:].sort_values(ascending=False)[:fill].index.values.tolist()
        portfolio=portfolio+new_picks
        print(portfolio)
    monthly_ret_df=pd.DataFrame(np.array(monthly_ret),columns=["mon_ret"])
    return monthly_ret_df

In [ ]:
CAGR(pflio(return_df,6,3))

In [ ]:
sharpe(pflio(return_df,6,3),0.025)

In [ ]:
max_dd(pflio(return_df,6,3))

In [ ]:
DJI=pdr.get_data_yahoo("^DJI",datetime.date.today().datetime.timedelta(1900),datetime.date.today())

In [ ]:
DJI["mon_ret"=DJI["Adj Close"].pct_change()

In [ ]:
CAGR(DJI)

In [ ]:
sharpe(DJI,0.025)

In [ ]:
max_dd(DJI)

In [ ]:
fig,ax=plt.subplots()
plt.plot((1+pflio(return_df,6,3)).cumprod())
plt.plot((1+DJI["mon_ret"][2:].reset_index(drop=True)).cumprod())
plt.title("Index Return vs Strategy Return")
plt.ylabel("cumulative return")
plt.xlabel("months")
ax.legend(["Strategy Return","Index Return"])